In [ ]:
import datetime
import os
import sys
import importlib
from itertools import chain
from sklearn.decomposition import PCA

import numpy as np
import keras
import tensorflow as tf

In [ ]:
def run_experiment(
    model, 
    inputs,
    labels,
    val_inputs,
    val_labels,
    test_inputs=None,
    test_labels=None,
    weight_decay=0.004, 
    label_smoothing=False, 
    recall_threshold=0.8, 
    save_path='.', 
    batch_size=512,
    num_epochs=10,
    learning_rate=1e-3, 
    fit_model=True
):

    # lr_schedule = keras.optimizers.schedules.PiecewiseConstantDecay(
    #     boundaries=[200, 500, 1000,],
    #     values=[learning_rate, learning_rate * 0.1, learning_rate * 0.01, learning_rate * 0.001]
    # )
    
    optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate,
        weight_decay=weight_decay,
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(from_logits=False, 
                                             name="binary_crossentropy", label_smoothing=label_smoothing),
        metrics=[
            keras.metrics.AUC(from_logits=False, name='auc_roc'),
            keras.metrics.PrecisionAtRecall(recall_threshold, num_thresholds=200, name='p_at_r'),
        ],
    )


    if fit_model:
        model_checkpoint = keras.callbacks.ModelCheckpoint(
            filepath=f"{save_path}/{datetime.datetime.now().strftime('%d%m_%H%M')}" + '/' + "{epoch:02d}-{val_loss:.3f}-{val_auc_roc:.3f}.tf",
            save_weights_only=False,
            monitor='val_auc_roc',
            save_best_only=True,
            mode='max'

        )
        
        reduce_lr = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=5
        )
        
        early_stopping = keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True
        )

        history = model.fit(
            x=inputs,
            y=labels,
            batch_size=batch_size,
            epochs=num_epochs,
            # validation_split=0.15,
            validation_data=(val_inputs, val_labels),
            callbacks=[model_checkpoint, early_stopping, reduce_lr],
            validation_freq=1,
            shuffle=True
        )

        if test_inputs is not None:
            _, auc_roc, p_at_r = model.evaluate(test_inputs, test_labels)
            print(f"Test auc_roc: {round(auc_roc * 100, 3)}%")
            print(f"Test p_at_r: {round(p_at_r * 100, 3)}%")
        
        return history
    return

In [ ]:
def get_dense_layer_block(
    name_tag, 
    output_dim, 
    add_dropout=True, 
    add_batch_norm=True, 
    activation_fn='relu',
    dropout_prob=0.25
):
    dense_model = keras.Sequential(
        keras.layers.Dense(output_dim, activation=None, name=f"{name_tag}_dense")
    )
    if activation_fn is not None:
        keras_layers = importlib.import_module('tensorflow.keras.layers')
        activation_layer = getattr(keras_layers, activation_fn)
        dense_model.add(activation_layer())   
    if add_batch_norm:
        dense_model.add(
            keras.layers.BatchNormalization(name=f"{name_tag}_batch_norm")
        )
    if add_dropout:
        dense_model.add(
            keras.layers.Dropout(rate=dropout_prob, name=f"{name_tag}_dropout")
        )
    return dense_model

In [ ]:
class LoanStateModel(keras.Model):
    def __init__(self, num_dense_blocks=1, emb_dim=30, activation_fn='relu'):
        super().__init__()
        self.dense_blocks = keras.Sequential([get_dense_layer_block(f"loan_state_{i}", emb_dim, activation_fn=activation_fn) 
                             for i in range(1, num_dense_blocks + 1)])
        self.final_dense_block = get_dense_layer_block(
            "loan_state_final",
            emb_dim,
            activation_fn=None,
            add_batch_norm=False,
            add_dropout=False
        )
        
    def call(self, x):
        x = self.dense_blocks(x)
        x = self.final_dense_block(x)
        return x 

In [ ]:
class ActionModel(keras.Model):
    """
    This architecture use seperate emb for each action. if there are 6 actions and 4 time slots, there are 24 actions
    Hence, there will be 24 embeddings produced.
    """
    def __init__(self, num_actions, emb_dim=30, seq_len=6, num_dense_blocks=1, activation_fn='relu'):
        super().__init__()
        self.emb_layer = keras.layers.Embedding(input_dim=num_actions + 1, 
                                                output_dim=emb_dim, 
                                                mask_zero=True,
                                                input_length=seq_len)
        
        self.dense_blocks = keras.Sequential([get_dense_layer_block(f"action_{i}", emb_dim, activation_fn=activation_fn) 
                             for i in range(1, num_dense_blocks + 1)])
        self.final_dense_block = get_dense_layer_block(
            "action_final",
            emb_dim,
            activation_fn=None,
            add_batch_norm=False,
            add_dropout=False
        )
        
    def call(self, x):
        x = self.emb_layer(x)
        x = tf.math.reduce_mean(x, axis=1, name='action_mean_pool')
        x = self.dense_blocks(x)
        x = self.final_dense_block(x)
        return x

In [ ]:
class ActionModelSepEmb(keras.Model):
    """
    This architecture use seperate emb for ch and time and then concatenates to get action embedding.
    """
    def __init__(self, num_channels=6, num_time_slots=5, emb_dim=30, seq_len=6, num_dense_blocks=1, activation_fn='relu'):
        super().__init__()
        self.channel_emb_layer = keras.layers.Embedding(input_dim=num_channels + 1, 
                                                output_dim=emb_dim, 
                                                mask_zero=True,
                                                input_length=seq_len)
        
        self.time_emb_layer = keras.layers.Embedding(input_dim=num_time_slots + 1, 
                                                output_dim=emb_dim, 
                                                mask_zero=True,
                                                input_length=seq_len)
        self.concat_layer = keras.layers.Concatenate(axis=-1)
        self.dense_blocks = keras.Sequential([get_dense_layer_block(f"action_{i}", emb_dim, activation_fn=activation_fn) 
                             for i in range(1, num_dense_blocks + 1)])
        self.final_dense_block = get_dense_layer_block(
            "action_final",
            emb_dim,
            activation_fn=None,
            add_batch_norm=False,
            add_dropout=False
        )
        
    def call(self, inputs):
        ch_inputs, time_inputs = inputs
        ch_emb = self.channel_emb_layer(ch_inputs) 
        time_emb = self.time_emb_layer(time_inputs)
        ch_time_concat = self.concat_layer([ch_emb, time_emb])
        x = tf.math.reduce_mean(ch_time_concat, axis=1, name='action_mean_pool')
        x = self.dense_blocks(x)
        x = self.final_dense_block(x)
        return x

In [ ]:
class LoanStateActionModel(keras.Model):
    def __init__(self, loan_state_model_config, action_model_config, sep_ch_time_emb=True):
        super().__init__()
        self.loan_state_model = LoanStateModel(**loan_state_model_config)
        if sep_ch_time_emb:
            self.action_model = ActionModelSepEmb(**action_model_config)
        else:
            self.action_model = ActionModel(**action_model_config)
        self.cosine_layer = keras.layers.Dot(axes=-1, normalize=True)
        
    def call(self, inputs):
        x1, x2 = inputs
        loan_state_output = self.loan_state_model(x1)
        action_output = self.action_model(x2)
        x = self.cosine_layer([loan_state_output, action_output])
        x = (x + 1) / 2
        return x

In [ ]:
class LoanStateActionContrastiveModel(keras.Model):
    """
    Use contrastive learning on only positive samples
    """
    def __init__(self, loan_state_model_config, action_model_config, sep_ch_time_emb=True, scale=5):
        super().__init__()
        self.scale = scale
        self.loan_state_model = LoanStateModel(**loan_state_model_config)
        if sep_ch_time_emb:
            self.action_model = ActionModelSepEmb(**action_model_config)
        else:
            self.action_model = ActionModel(**action_model_config)
        self.cosine_layer = keras.layers.Dot(axes=-1, normalize=True)
        
    def call(self, inputs):
        x1, x2 = inputs
        loan_state_output = self.loan_state_model(x1)
        action_output = self.action_model(x2)
        loan_state_output = tf.expand_dims(loan_state_output, axis=0)
        action_output = tf.expand_dims(action_output, axis=0)
        x = self.cosine_layer([loan_state_output, action_output])
        x = tf.squeeze(x, axis=0) * self.scale
        # to reduce false negatives
        tp = tf.linalg.diag_part(x)
        mask = tf.cast(tf.equal(x, tf.expand_dims(tp, axis=1)), dtype=tf.float32)
        fill = tf.zeros(mask.shape[0])
        mask = tf.linalg.set_diag(mask, fill)
        x = x * (1 - mask)
        return x

In [ ]:
loan_state_action_model = LoanStateActionModel({'num_dense_blocks': 2, 'activation_fn': 'LeakyReLU'}, {'activation_fn': 'LeakyReLU'}, sep_ch_time_emb=True)

In [ ]:
hisotry = run_experiment(
    model=loan_state_action_model,
    inputs=(x_train_state, (x_train_channels, x_train_times)),
    labels=y_train,
    val_inputs=(x_val_state, (x_val_channels, x_val_times)),
    val_labels=y_val,
    test_inputs=(x_test_state, (x_test_channels, x_test_times)),
    test_labels=y_test,
    save_path='trained_models/',
)